<p align="center"><img src="../Additionals/Empa-Workshops-Template-Banner2.png" alt="University Workshops" style="display: block; margin: 0 auto" height=/></p>

## Accelerator Workshops'a Hoşgeldiniz!

Empa Electronics tarafından düzenlenen Accelerators Workshops etkinlikleri serimizin Uçta Yapay Zeka adımına hoşgeldiniz. Bu açık-kaynak repository, workshop etkiliğimizde deneyimleyeceğiniz "ST Platformları İçin Edge-AI Çözümleri Geliştirme" uygulaması çalışma ortamını edinebilmeniz ve aktivitelere kolaylıkla eşlik edebilmeniz için sizinle paylaşılmıştır.

Bu script, uygulama içeriğinde geliştirilen modelin testi için oluşturulmuştur.
  

**Uygulama Adımları:**

1. Gereksinimlerin Dahil Edilmesi

2. Eğitilmiş Modelin Yüklenmesi

3. Seri Port ile Veri Örneği Okuma & Eğitilmiş Modeli Kullanarak Tahminleme

## 1. Gereksinimlerin Dahil Edilmesi

Gerekli modeller için import işlemi:

In [1]:
import tensorflow as tf
import serial
import numpy as np

2024-06-06 15:52:26.114092: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-06 15:52:26.148306: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-06 15:52:26.148606: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 15:52:26.806066: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## 2. Eğitilmiş Modelin Yüklenmesi

Loading the trained model:

In [2]:
import os
import re

def get_largest_performance_model(directory):
    pattern = re.compile(r"Model_CNN_Hand_Character_Recognition_0_(\d+)\.h5")

    max_performance = -1
    best_model = None

    for filename in os.listdir(directory):
        match = pattern.match(filename)
        if match:
            performance = int(match.group(1))
            if performance > max_performance:
                max_performance = performance
                best_model = filename

    return best_model

directory = "./Models"
name_best_model = get_largest_performance_model(directory)
print(f"The model with the best performance is: {name_best_model}")

The model with the best performance is: Model_CNN_Hand_Character_Recognition_0_9246.h5


In [3]:
model = tf.keras.models.load_model(f"Models/{name_best_model}")

2024-06-06 15:52:29.701543: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-06 15:52:29.737348: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Defining the class mapping dictionary:

In [4]:
# class ID to class name mapping
classes = {0: "CIRCLE",
           1: "HORIZONTAL",
           2: "STANDBY",
           3: "TRIANGLE",
           4: "VERTICAL"}

## 3. Seri Port ile Veri Örneği Okuma & Eğitilmiş Modeli Kullanarak Tahminleme

Seri port konfigürasyonu için gerekli değerlerin tanımlanması:

In [5]:
serial_port = '/dev/ttyACM0'   # Ubuntu
# serial_port = 'COM0'             # Windows
baud_rate = 115200
time_out = 2

Seri port üzerinden veri okuma & okunan veri örneklerinin tahminlenmesi

In [ ]:
# collect data, get inference from model
while True:
    # open the serial port, set baudrate, set timeout
    with serial.Serial(serial_port, baud_rate, timeout=time_out) as ser_read:
        # read the line from serial port
        x = ser_read.readline()
        # parse the line into numpy array
        line = np.array(str(x).replace("b'", "").replace("\\n'", "").replace("\\r", "").split(" ")[:-1])
        # ignore the line if the length is not 6*128=768
        if len(line) == 768:
            # convert the array of strings to array of floating point numbers
            line = line.astype(np.float32)
            # reshape the array for model
            line = line.reshape(128,6)
            # add a batch shape to the array
            line = np.expand_dims(line, axis=0)
            # get the model output
            out = model(line)
            # model outputs a tensor, convert it to numpy array
            out = np.array(out)
            # get the highest scoring output as predicted class
            class_index = np.argmax(out)
            # get the class name from dict
            class_name = classes[class_index]
            # print the results
            print(f"\rProbabilities -> CIRCLE: {out[0,0]:.2f} HORIZONTAL: {out[0,1]:.2f} STANDBY: {out[0,2]:.2f} TRIANGLE: {out[0,3]:.2f} VERTICAL: {out[0,4]:.2f} --- Prediction: {class_name} {' '*50}",end="")